Edit variables_to_edit.py before running

In [1]:
from random import randint
from tqdm import tqdm
from time import sleep

import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC


from config import username, password, url
from variables_to_edit import *

In [2]:
# Setup browsing environment

response = requests.get(url)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [3]:
# Functions to interact with unique challenges of the website
def click_element(method, target):
    for _ in range(10):
        try:
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((method, target)))
            element.click()
            break
        except:
            sleep(1)

def present_element(method, target, input):
    for _ in range(10):
        try:
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((method, target)))
            element.send_keys(input)
            break
        except:
            sleep(1)
    # return element

def all_elements(method, target):
    elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((method, target)))
    return elements

def invisible_wait(method, target):
    elements = WebDriverWait(driver, 180).until(EC.invisibility_of_element_located((method, target)))
    return elements

# Website navigation functions
def login(username, password):
    present_element(By.ID, 'user_email', username)
    present_element(By.ID, 'user_password', password)
    click_element(By.NAME, 'commit')

def get_link_texts(company, closed_clinics):
    click_element(By.ID, 'user-usurp')
    options = all_elements(By.PARTIAL_LINK_TEXT, company)

    link_texts = [option.text for option in options]
    link_texts = link_texts[1:-4]  # removes hq and deactivated clinics

    for closed in closed_clinics:
        closed = company + ' ' + closed
        link_texts = [text for text in link_texts if text != closed]

    return link_texts

# The below can be used outside of a loop as well
#Clean up
def remove_previous_lists(driver, recipient, recipient_list):
    try:
        click_element(By.LINK_TEXT, 'CRM')
    except:
        driver.refresh()
        click_element(By.LINK_TEXT, 'CRM')
    
    click_element(By.LINK_TEXT, 'Recipient Lists')
    try:
        email_list = all_elements(By.XPATH, f'//optgroup[@label="{recipient.title()}"]/option')
    except:
        pass
    
    length = len(email_list)
    for _ in tqdm(range(0,length)):
        if driver.find_element(By.ID, 'select_show_recipients_list').text != recipient_list:
            click_element(By.LINK_TEXT, 'Delete List')
            click_element(By.ID, 'alertify-ok')
            try:
                invisible_wait(By.CSS_SELECTOR, ".loader.hide")
            except:
                driver.refresh()
            sleep(.5)

# Build lists
def patient_recipient_load(driver, recipient, three_months, recipient_list):
    click_element(By.PARTIAL_LINK_TEXT, recipient.title())
    click_element(By.CLASS_NAME, 'filter-button')

    datepicker = all_elements(By.CSS_SELECTOR, ".datepicker.patient_filter_to")[-1]
    driver.execute_script(f"arguments[0].value = '{three_months}';", datepicker)

    sleep(.5)

    filter_button = all_elements(By.XPATH, "//input[@name='commit' and @type='submit' and @value='Filter']")[-1]
    filter_button.click()

    invisible_wait(By.CSS_SELECTOR, ".lead_target_loader")

    click_element(By.CLASS_NAME, 'select_all_patients')
    click_element(By.LINK_TEXT, 'Select Bulk Action')

    try:
        click_element(By.CLASS_NAME, 'create_recipients_list')
    except:
        click_element(By.LINK_TEXT, 'Select Bulk Action')
        click_element(By.CLASS_NAME, 'create_recipients_list')

    click_element(By.ID, 'recipients_list_name')
    name_list = present_element(By.ID, 'recipients_list_name')
    name_list.send_keys(recipient_list)

    click_element(By.CSS_SELECTOR, 'input[name="commit"][role="create_list"]')

    invisible_wait(By.CSS_SELECTOR, ".lead_target_loader")
    sleep(3)
    try:
        close = all_elements(By.XPATH, "//div[@class='hide overlay patient']/div[@class='close']")[-1]
        close.click()
    except:
        pass

    # rj doesn't always load the email list we create. it takes a while to think.
    sleep(3)

# Finalize campaign
def create_campaign(driver, email_campaign_name, subject, email_body, recipient_list):
    try:
        click_element(By.LINK_TEXT, 'CRM')
    except:
        driver.refresh()
        click_element(By.LINK_TEXT, 'CRM')

    click_element(By.LINK_TEXT, 'Campaigns')
    click_element(By.LINK_TEXT, 'Create Campaign')
    click_element(By.LINK_TEXT, 'Create A New Campaign')

    present_element(By.ID,'campaign_name', email_campaign_name)

    for _ in range(5):
        try:
            type = Select(driver.find_element(By.ID, 'campaign_campaign_type'))
            if recipient == 'patient':
                type.select_by_visible_text(recipient.title())
            else:
                type.select_by_visible_text('Provider')
            break 
        except:
            sleep(1)

    click_element(By.ID,'create_template')
    click_element(By.LINK_TEXT, 'Simple Email')

    present_element(By.ID, 'email_template_name', email_campaign_name)

    present_element(By.ID, 'email_template_subject', subject)
        
    present_element(By.ID, 'email_template_reply_to', '{{practice_email}}')

    iframe = driver.find_element(By.ID, "email_template_body_ifr")
    driver.switch_to.frame(iframe)

    body = driver.find_element(By.ID, 'tinymce')

    driver.execute_script("arguments[0].innerHTML = arguments[1];", body, email_body)
    driver.switch_to.default_content()

    click_element(By.CLASS_NAME, 'next_stage')

    click_element(By.ID, 'time_from')
    present_element(By.ID, 'time_from', str(randint(0, 2)))

    present_element(By.ID, 'event', 'Email Creation')

    click_element(By.CLASS_NAME, 'next_stage')

    for _ in range(5):
        try:
            lists = Select(driver.find_element(By.ID, 'lists'))
            lists.select_by_visible_text(recipient_list)
            break 
        except:
            sleep(1)

    click_element(By.CLASS_NAME, 'next_stage')
    sleep(3)
    click_element(By.CLASS_NAME, 'next_stage')

    click_element(By.CLASS_NAME, "dropdown-toggle")

In [4]:
login(username,password)
link_texts = get_link_texts(company, closed_clinics)

In [ ]:
# link_texts = link_texts[:-0] #for broken loops
try:
    link_texts.remove('') # in the event of a more personalized email
except:
    pass

# Main function
for clinic in link_texts[::-1]:
    try:
        driver.find_element(By.LINK_TEXT, clinic).click()
    except:
        click_element(By.CLASS_NAME, "dropdown-toggle")
        click_element(By.LINK_TEXT, clinic)
    print(clinic)
    
    if recipient == 'patient':
        patient_recipient_load(driver, recipient, three_months, recipient_list)
    else:
        #build active function
        break
    
    remove_previous_lists(driver, recipient, recipient_list)
    create_campaign(driver, email_campaign_name, subject, email_body, recipient_list)
    click_element(By.CLASS_NAME, "dropdown-toggle")

In [8]:
# Single Clinic Load
# patient_recipient_load(driver, recipient, three_months, recipient_list)
# remove_previous_lists(driver, recipient, recipient_list)

create_campaign(driver, email_campaign_name, subject, email_body, recipient_list)